 # <center> 新冠病毒疫情可视化分析 </center>

## Part1 全国疫情数据趋势

### Step1 获取实时数据

In [2]:
# -*- coding: utf-8 -*-
import time, json, requests
from datetime import datetime

#------------------------------------------------------------------------------
# 第一步 抓取实时json数据
# 参考博客文章 https://blog.csdn.net/xufive/article/details/104093197
#------------------------------------------------------------------------------
def catch_daily():
    url = 'https://view.inews.qq.com/g2/getOnsInfo?name=wuwei_ww_cn_day_counts&callback=&_=%d'%int(time.time()*1000)
    data = json.loads(requests.get(url=url).json()['data'])
    data.sort(key=lambda x:x['date'])
    
    date_list = list()        # 日期
    confirm_list = list()     # 确诊
    suspect_list = list()     # 疑似
    dead_list = list()        # 死亡
    heal_list = list()        # 治愈
    for item in data:
        month, day = item['date'].split('/')
        date_list.append(datetime.strptime('2020-%s-%s'%(month, day), '%Y-%m-%d'))
        confirm_list.append(int(item['confirm']))
        suspect_list.append(int(item['suspect']))
        dead_list.append(int(item['dead']))
        heal_list.append(int(item['heal']))
    return date_list, confirm_list, suspect_list, dead_list, heal_list

#------------------------------------------------------------------------------
# 第二步 存储数据至CSV文件
#------------------------------------------------------------------------------
def load_csv():
    # 获取数据
    date_list, confirm_list, suspect_list, dead_list, heal_list = catch_daily()
    print(date_list)                        # 日期
    print(confirm_list)                     # 确诊数据
    print(suspect_list)                     # 疑似数据
    print(dead_list)                        # 死亡数据
    print(heal_list)                        # 治愈数据

    # 获取当前日期命名eg.(2020-06-19-daily.csv)
    n = time.strftime("%Y-%m-%d") + "-daily.csv"
    fw = open(n, 'w', encoding='utf-8')
    fw.write('date,confirm,suspect,dead,heal\n')
    i = 0
    while i<len(date_list):
        date = str(date_list[i].strftime("%Y-%m-%d"))
        fw.write(date+','+str(confirm_list[i])+','+str(suspect_list[i])+','+str(dead_list[i])+','+str(heal_list[i])+'\n')
        i = i + 1
    else:
        print("Over write file!")
        fw.close()

# 主函数
if __name__ == '__main__':
    load_csv()


[datetime.datetime(2020, 1, 13, 0, 0), datetime.datetime(2020, 1, 14, 0, 0), datetime.datetime(2020, 1, 15, 0, 0), datetime.datetime(2020, 1, 16, 0, 0), datetime.datetime(2020, 1, 17, 0, 0), datetime.datetime(2020, 1, 18, 0, 0), datetime.datetime(2020, 1, 19, 0, 0), datetime.datetime(2020, 1, 20, 0, 0), datetime.datetime(2020, 1, 21, 0, 0), datetime.datetime(2020, 1, 22, 0, 0), datetime.datetime(2020, 1, 23, 0, 0), datetime.datetime(2020, 1, 24, 0, 0), datetime.datetime(2020, 1, 25, 0, 0), datetime.datetime(2020, 1, 26, 0, 0), datetime.datetime(2020, 1, 27, 0, 0), datetime.datetime(2020, 1, 28, 0, 0), datetime.datetime(2020, 1, 29, 0, 0), datetime.datetime(2020, 1, 30, 0, 0), datetime.datetime(2020, 1, 31, 0, 0), datetime.datetime(2020, 2, 1, 0, 0), datetime.datetime(2020, 2, 2, 0, 0), datetime.datetime(2020, 2, 3, 0, 0), datetime.datetime(2020, 2, 4, 0, 0), datetime.datetime(2020, 2, 5, 0, 0), datetime.datetime(2020, 2, 6, 0, 0), datetime.datetime(2020, 2, 7, 0, 0), datetime.datetime(

### Step2 画折线面积图

In [3]:
# -*- coding: utf-8 -*-
import time, json, requests
from datetime import datetime
import pandas as pd
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.commons.utils import JsCode

#-------------------------------------------------------------------------------------
# 第一步：读取数据
#-------------------------------------------------------------------------------------
n = time.strftime("%Y-%m-%d") + "-daily.csv"
data = pd.read_csv(n)
date_list = list(data['date'])
confirm_list = list(data['confirm'])
suspect_list = list(data['suspect'])
dead_list = list(data['dead'])
heal_list = list(data['heal'])
print(date_list)                        # 日期
print(confirm_list)                     # 确诊数据
print(suspect_list)                     # 疑似数据
print(dead_list)                        # 死亡数据
print(heal_list)                        # 治愈数据


#-------------------------------------------------------------------------------------
# 第二步：绘制折线面积图
#-------------------------------------------------------------------------------------
line = (
    Line()
    .add_xaxis(date_list)
    # 平均线 最大值 最小值
    .add_yaxis('确诊数据', confirm_list, is_smooth=True,
               markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")]),
               markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max"),
                                                       opts.MarkPointItem(type_="min")]))
    .add_yaxis('疑似数据', suspect_list, is_smooth=True,
               markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")]),
               markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max"),
                                                       opts.MarkPointItem(type_="min")]))
    .add_yaxis('死亡数据', dead_list, is_smooth=True,
               markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")]),
               markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max"),
                                                       opts.MarkPointItem(type_="min")]))
    .add_yaxis('治愈数据', heal_list, is_smooth=True,
               markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")]),
               markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max"),
                                                       opts.MarkPointItem(type_="min")]))
    # 隐藏数字 设置面积
    .set_series_opts(
        areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        label_opts=opts.LabelOpts(is_show=False))
    # 设置x轴标签旋转角度
    .set_global_opts(xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-30)), 
                     yaxis_opts=opts.AxisOpts(name='人数', min_=3), 
                     title_opts=opts.TitleOpts(title='2019-nCoV曲线图'))        
    )

line.render_notebook()

['2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-18', '2020-01-19', '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23', '2020

D:\soft\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


## Part2 全国各地疫情地图

### Step1 获取实时数据

In [4]:
# -*- coding: utf-8 -*-
#------------------------------------------------------------------------------
# 第一步：抓取数据
#------------------------------------------------------------------------------
import time, json, requests

# 抓取实时json数据
url = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5&callback=&_=%d'%int(time.time()*1000)
data = json.loads(requests.get(url=url).json()['data'])
print(data)
print(data.keys())

# 统计省份信息(34个省份 湖北 广东 河南 浙江 湖南 安徽....)
num = data['areaTree'][0]['children']
print(len(num))
for item in num:
    print(item['name'],end=" ")   # 不换行
else:
    print("\n")                     # 换行
    
# 显示湖北省数据
hubei = num[0]['children']
for item in hubei:
    print(item)
else:
    print("\n")

# 解析确诊数据
total_data = {}
for item in num:
    if item['name'] not in total_data:
        total_data.update({item['name']:0})
    for city_data in item['children']:
        total_data[item['name']] +=int(city_data['total']['confirm'])    
print(total_data)
# {'湖北': 48206, '广东': 1241, '河南': 1169, '浙江': 1145, '湖南': 968, ...,  '澳门': 10, '西藏': 1}

# 解析疑似数据
total_suspect_data = {}
for item in num:
    if item['name'] not in total_suspect_data:
        total_suspect_data.update({item['name']:0})
    for city_data in item['children']:
        total_suspect_data[item['name']] +=int(city_data['total']['suspect'])    
print(total_suspect_data)

# 解析死亡数据
total_dead_data = {}
for item in num:
    if item['name'] not in total_dead_data:
        total_dead_data.update({item['name']:0})
    for city_data in item['children']:
        total_dead_data[item['name']] +=int(city_data['total']['dead'])    
print(total_dead_data)

# 解析治愈数据
total_heal_data = {}
for item in num:
    if item['name'] not in total_heal_data:
        total_heal_data.update({item['name']:0})
    for city_data in item['children']:
        total_heal_data[item['name']] +=int(city_data['total']['heal'])    
print(total_heal_data)

# 解析新增确诊数据
total_new_data = {}
for item in num:
    if item['name'] not in total_new_data:
        total_new_data.update({item['name']:0})
    for city_data in item['children']:
        total_new_data[item['name']] +=int(city_data['today']['confirm']) # today    
print(total_new_data)

#------------------------------------------------------------------------------
# 第二步：存储数据至CSV文件
#------------------------------------------------------------------------------
names = list(total_data.keys())          # 省份名称
num1 = list(total_data.values())         # 确诊数据
num2 = list(total_suspect_data.values()) # 疑似数据(全为0)
num3 = list(total_dead_data.values())    # 死亡数据
num4 = list(total_heal_data.values())    # 治愈数据
num5 = list(total_new_data.values())     # 新增确诊病例
print(names)
print(num1)
print(num2)
print(num3)
print(num4)
print(num5)

# 获取当前日期命名(2020-02-13-all.csv)
n = time.strftime("%Y-%m-%d") + "-china.csv"
fw = open(n, 'w', encoding='utf-8')
fw.write('province,confirm,dead,heal,new_confirm\n')
i = 0
while i<len(names):
    fw.write(names[i]+','+str(num1[i])+','+str(num3[i])+','+str(num4[i])+','+str(num5[i])+'\n')
    i = i + 1
else:
    print("Over write file!")
    fw.close()


{'lastUpdateTime': '2020-06-19 17:49:15', 'chinaTotal': {'confirm': 84943, 'heal': 79951, 'dead': 4645, 'nowConfirm': 347, 'suspect': 7, 'nowSevere': 13, 'importedCase': 1864, 'noInfect': 110}, 'chinaAdd': {'confirm': 3, 'heal': 2, 'dead': 0, 'nowConfirm': 1, 'suspect': 0, 'nowSevere': 0, 'importedCase': 0, 'noInfect': 5}, 'isShowAdd': False, 'showAddSwitch': {'all': False, 'confirm': True, 'suspect': True, 'dead': True, 'heal': True, 'nowConfirm': True, 'nowSevere': True, 'importedCase': True, 'noInfect': True}, 'areaTree': [{'name': '中国', 'today': {'confirm': 3, 'isUpdated': True}, 'total': {'nowConfirm': 347, 'confirm': 84943, 'suspect': 7, 'dead': 4645, 'deadRate': '5.47', 'showRate': False, 'heal': 79951, 'healRate': '94.12', 'showHeal': True}, 'children': [{'name': '北京', 'today': {'confirm': 0, 'confirmCuts': 0, 'isUpdated': False, 'tip': '北京累计报告境外输入确诊病例174例。\n北京卫健委未明确治愈病例和死亡病例的归属分区，北京分区现有确诊数据暂无法获取。'}, 'total': {'nowConfirm': 184, 'confirm': 777, 'suspect': 0, 'dead': 9, 'deadRat

### Part2 绘制地图

In [5]:
# -*- coding: utf-8 -*-
import time, json, requests
import pandas as pd
from pyecharts.charts import Map
import pyecharts.options as opts

#-------------------------------------------------------------------------------------
# 第一步：读取数据
#-------------------------------------------------------------------------------------
n = time.strftime("%Y-%m-%d") + "-china.csv"
data = pd.read_csv(n)
list_data = list(zip(list(data['province']), list(data['confirm'])))
print(list_data)
# [('湖北', 48206), ('广东', 1241), ('河南', 1169), ('浙江', 1145), ..., ('澳门', 10), ('西藏', 1)]

#-------------------------------------------------------------------------------------
# 第二步：绘制全国地图
# 参考文章：https://blog.csdn.net/shineych/article/details/104231072 [shineych大神]
#-------------------------------------------------------------------------------------
def map_cn_disease_dis() -> Map:
    c = (
        Map()
        .add('中国', list_data, 'china')
        .set_global_opts(
            title_opts=opts.TitleOpts(title='全国地图（确诊数）'),
            visualmap_opts=opts.VisualMapOpts(is_show=True,
                                              split_number=6,
                                              is_piecewise=True,  # 是否为分段型
                                              pos_top='center',
                                              pieces=[
                                                   {'min': 10000, 'color': '#7f1818'},  #不指定 max
                                                   {'min': 1000, 'max': 10000},
                                                   {'min': 500, 'max': 999},
                                                   {'min': 100, 'max': 499},
                                                   {'min': 10, 'max': 99},
                                                   {'min': 0, 'max': 5} ],                                              
                                              ),
        )
    )
    return c

#map_cn_disease_dis().render('全国地图.html')
map_cn_disease_dis().render_notebook()

[('北京', 777), ('香港', 1127), ('上海', 697), ('广东', 1631), ('四川', 589), ('甘肃', 151), ('河北', 340), ('陕西', 314), ('台湾', 446), ('福建', 362), ('辽宁', 153), ('内蒙古', 238), ('重庆', 582), ('天津', 197), ('山东', 792), ('海南', 171), ('浙江', 1269), ('河南', 1276), ('新疆', 76), ('湖南', 1019), ('吉林', 155), ('江西', 932), ('澳门', 45), ('西藏', 1), ('云南', 185), ('青海', 18), ('广西', 254), ('山西', 198), ('贵州', 147), ('安徽', 991), ('江苏', 653), ('宁夏', 75), ('湖北', 68135), ('黑龙江', 947)]


D:\soft\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


## Part3 北京疫情情况

### Step1 获取数据

In [8]:
# -*- coding: utf-8 -*-
# By：Easmount CSDN xiuzhang

#------------------------------------------------------------------------------
# 第一步：抓取数据
#------------------------------------------------------------------------------
import time, json, requests

# 抓取实时json数据
url = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5&callback=&_=%d'%int(time.time()*1000)
data = json.loads(requests.get(url=url).json()['data'])
print(data)
print(data.keys())

# 统计省份信息(34个省份 湖北 广东 河南 浙江 湖南 安徽....)
num = data['areaTree'][0]['children']
print(len(num))

# 获取北京下标
k = 0
for item in num:
    print(item['name'],end=" ")   # 不换行
    if item['name'] in "北京":
        print("")
        print(item['name'], k)
        break
    k = k + 1
print("") # 换行

# 显示北京市数据
bj = num[k]['children']
for item in bj:
    print(item)
else:
    print("\n")

#------------------------------------------------------------------------------
# 第二步：解析数据
#------------------------------------------------------------------------------
# 解析确诊数据
total_data = {}
for item in bj:
    if item['name'] not in total_data:
        total_data.update({item['name']:0})
    total_data[item['name']] = item['total']['confirm']
print('确诊人数')
print(total_data)


# 解析疑似数据
total_suspect_data = {}
for item in bj:
    if item['name'] not in total_suspect_data:
        total_suspect_data.update({item['name']:0})
    total_suspect_data[item['name']] = item['total']['suspect']    
print('疑似人数')
print(total_suspect_data)

# 解析死亡数据
total_dead_data = {}
for item in bj:
    if item['name'] not in total_dead_data:
        total_dead_data.update({item['name']:0})
    total_dead_data[item['name']] = item['total']['dead']  
print('死亡人数')
print(total_dead_data)

# 解析治愈数据
total_heal_data = {}
for item in bj:
    if item['name'] not in total_heal_data:
        total_heal_data.update({item['name']:0})
    total_heal_data[item['name']] = item['total']['heal']
print('治愈人数')
print(total_heal_data)

# 解析新增确诊数据
total_new_data = {}
for item in bj:
    if item['name'] not in total_new_data:
        total_new_data.update({item['name']:0})
    total_new_data[item['name']] = item['today']['confirm'] # today    
print('新增确诊人数')
print(total_new_data)

#------------------------------------------------------------------------------
# 第三步：存储数据至CSV文件
#------------------------------------------------------------------------------
names = list(total_data.keys())          # 省份名称
num1 = list(total_data.values())         # 确诊数据
num2 = list(total_suspect_data.values()) # 疑似数据(全为0)
num3 = list(total_dead_data.values())    # 死亡数据
num4 = list(total_heal_data.values())    # 治愈数据
num5 = list(total_new_data.values())     # 新增确诊病例
print(names)
print(num1)
print(num2)
print(num3)
print(num4)
print(num5)

# 获取当前日期命名(2020-02-13-gz.csv)
n = time.strftime("%Y-%m-%d") + "-bj.csv"
fw = open(n, 'w', encoding='utf-8')
fw.write('province,confirm,dead,heal,new_confirm\n')
i = 0
while i<len(names):
    fw.write(names[i]+','+str(num1[i])+','+str(num3[i])+','+str(num4[i])+','+str(num5[i])+'\n')
    i = i + 1
else:
    print("Over write file!")
    fw.close()


{'lastUpdateTime': '2020-06-19 17:49:15', 'chinaTotal': {'confirm': 84943, 'heal': 79951, 'dead': 4645, 'nowConfirm': 347, 'suspect': 7, 'nowSevere': 13, 'importedCase': 1864, 'noInfect': 110}, 'chinaAdd': {'confirm': 3, 'heal': 2, 'dead': 0, 'nowConfirm': 1, 'suspect': 0, 'nowSevere': 0, 'importedCase': 0, 'noInfect': 5}, 'isShowAdd': False, 'showAddSwitch': {'all': False, 'confirm': True, 'suspect': True, 'dead': True, 'heal': True, 'nowConfirm': True, 'nowSevere': True, 'importedCase': True, 'noInfect': True}, 'areaTree': [{'name': '中国', 'today': {'confirm': 3, 'isUpdated': True}, 'total': {'nowConfirm': 347, 'confirm': 84943, 'suspect': 7, 'dead': 4645, 'deadRate': '5.47', 'showRate': False, 'heal': 79951, 'healRate': '94.12', 'showHeal': True}, 'children': [{'name': '北京', 'today': {'confirm': 0, 'confirmCuts': 0, 'isUpdated': False, 'tip': '北京累计报告境外输入确诊病例174例。\n北京卫健委未明确治愈病例和死亡病例的归属分区，北京分区现有确诊数据暂无法获取。'}, 'total': {'nowConfirm': 184, 'confirm': 777, 'suspect': 0, 'dead': 9, 'deadRat

### Step2 画疫情地图

In [6]:
# -*- coding: utf-8 -*-
import time, json, requests
import pandas as pd
from pyecharts.charts import Map
import pyecharts.options as opts

#-------------------------------------------------------------------------------------
# 第一步：读取数据
#-------------------------------------------------------------------------------------
n = time.strftime("%Y-%m-%d") + "-bj.csv"
data = pd.read_csv(n)
gz_data = list(zip(list(data['province']+'区'), list(data['confirm'])))
print(gz_data)

for a,b in gz_data:
    print(a, b, type(b))

#-------------------------------------------------------------------------------------
# 第二步：绘制贵州地图
#-------------------------------------------------------------------------------------
def map_bj_disease_dis() -> Map:
    c = (
        Map()
        .add('北京市', gz_data, '北京')
        .set_series_opts(label_opts = opts.LabelOpts(is_show=True, formatter='{b}\n{c}例'))
        .set_global_opts(
                    title_opts = opts.TitleOpts(title='北京市地图（确诊数）'),
                    visualmap_opts=opts.VisualMapOpts(is_show=True,
                                                      split_number=6,
                                                      is_piecewise=True,  # 是否为分段型
                                                      pos_top='center',
                                                      pieces=[
                                                           {'min': 50},  
                                                           {'min': 30, 'max': 49},
                                                           {'min': 20, 'max': 29},
                                                           {'min': 10, 'max': 19},
                                                           {'min': 1, 'max': 9},
                                                           {'value': 0, "label": '无确诊病例', "color": 'green'} ],                                              
                                                      ),
                )
    )
    return c
map_bj_disease_dis().render_notebook()

<class 'pandas.core.series.Series'>
[('丰台区', 174), ('大兴区', 67), ('海淀区', 71), ('东城区', 19), ('西城区', 57), ('房山区', 20), ('门头沟区', 5), ('境外输入区', 174), ('石景山区', 15), ('朝阳区', 76), ('地区待确认区', 1), ('外地来京区', 25), ('通州区', 19), ('昌平区', 29), ('密云区', 7), ('顺义区', 10), ('怀柔区', 7), ('延庆区', 1)]
丰台区 174 <class 'int'>
大兴区 67 <class 'int'>
海淀区 71 <class 'int'>
东城区 19 <class 'int'>
西城区 57 <class 'int'>
房山区 20 <class 'int'>
门头沟区 5 <class 'int'>
境外输入区 174 <class 'int'>
石景山区 15 <class 'int'>
朝阳区 76 <class 'int'>
地区待确认区 1 <class 'int'>
外地来京区 25 <class 'int'>
通州区 19 <class 'int'>
昌平区 29 <class 'int'>
密云区 7 <class 'int'>
顺义区 10 <class 'int'>
怀柔区 7 <class 'int'>
延庆区 1 <class 'int'>


D:\soft\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


## Part4 海外疫情地图

### Step1 获取数据

In [8]:
import requests
from datetime import datetime

def catch_data():
    """
    抓取当前实时数据，并返回 国家、大洲、确诊、疑似、死亡、治愈 列表
    :return:
    """
    url = 'https://api.inews.qq.com/newsqa/v1/automation/foreign/country/ranklist'
    data = requests.post(url).json()['data']

    date_list = list()  # 日期
    name_list = list() # 国家
    continent_list = list() # 大洲
    confirm_list = list()  # 确诊
    suspect_list = list()  # 疑似
    dead_list = list()  # 死亡
    heal_list = list()  # 治愈

    for item in data:
        month, day = item['date'].split('.')
        date_list.append(datetime.strptime('2020-%s-%s' % (month, day), '%Y-%m-%d'))
        name_list.append(item['name'])
        continent_list.append(item['continent'])
        confirm_list.append(item['confirm'])
        suspect_list.append(item['suspect'])
        dead_list.append(item['dead'])
        heal_list.append(item['heal'])

    return date_list, name_list, continent_list, confirm_list, suspect_list, dead_list, heal_list


def save_csv():
    """
    将数据存入 csv 文件
    :return:
    """
    date_list, name_list, continent_list, confirm_list, suspect_list, dead_list, heal_list = catch_data()
    fw = open('2019-nCoV.csv', 'w', encoding='utf-8')
    fw.write('date,name,continent,confirm,suspect,dead,heal\n')

    i = 0
    while i < len(date_list):
        date = str(date_list[i].strftime("%Y-%m-%d"))
        fw.write(date + ',' + str(name_list[i]) + ',' + str(continent_list[i]) + ',' + str(confirm_list[i]) + ',' + str(suspect_list[i]) + ',' + str(dead_list[i]) + ',' + str(heal_list[i]) + '\n')
        i = i + 1
    else:
        print("csv 写入完成")
        fw.close()


if __name__ == '__main__':
    save_csv()

csv 写入完成


### Step 2 画海外疫情地图

In [36]:
#定义国家中英文对照表
nameMap = {
        'Singapore Rep.':'新加坡',
        'Dominican Rep.':'多米尼加',
        'Palestine':'巴勒斯坦',
        'Bahamas':'巴哈马',
        'Timor-Leste':'东帝汶',
        'Afghanistan':'阿富汗',
        'Guinea-Bissau':'几内亚比绍',
        "Côte d'Ivoire":'科特迪瓦',
        'Siachen Glacier':'锡亚琴冰川',
        "Br. Indian Ocean Ter.":'英属印度洋领土',
        'Angola':'安哥拉',
        'Albania':'阿尔巴尼亚',
        'United Arab Emirates':'阿联酋',
        'Argentina':'阿根廷',
        'Armenia':'亚美尼亚',
        'French Southern and Antarctic Lands':'法属南半球和南极领地',
        'Australia':'澳大利亚',
        'Austria':'奥地利',
        'Azerbaijan':'阿塞拜疆',
        'Bahrain':'巴林',
        'Burundi':'布隆迪',
        'Belgium':'比利时',
        'Benin':'贝宁',
        'Burkina Faso':'布基纳法索',
        'Bangladesh':'孟加拉',
        'Bulgaria':'保加利亚',
        'The Bahamas':'巴哈马',
        'Bosnia and Herz.':'波斯尼亚和黑塞哥维那',
        'Belarus':'白俄罗斯',
        'Belize':'伯利兹',
        'Bermuda':'百慕大',
        'Bolivia':'玻利维亚',
        'Brazil':'巴西',
        'Brunei':'文莱',
        'Bhutan':'不丹',
        'Botswana':'博茨瓦纳',
        'Central African Rep.':'中非共和国',
        'Canada':'加拿大',
        'Switzerland':'瑞士',
        'Chile':'智利',
        'China':'中国',
        'Ivory Coast':'象牙海岸',
        'Cameroon':'喀麦隆',
        'Dem. Rep. Congo':'刚果（金）',
        'Congo':'刚果（布）',
        'Colombia':'哥伦比亚',
        'Costa Rica':'哥斯达黎加',
        'Cuba':'古巴',
        'N. Cyprus':'北塞浦路斯',
        'Cyprus':'塞浦路斯',
        'Czech Rep.':'捷克',
        'Germany':'德国',
        'Djibouti':'吉布提',
        'Denmark':'丹麦',
        'Algeria':'阿尔及利亚',
        'Ecuador':'厄瓜多尔',
        'Egypt':'埃及',
        'Eritrea':'厄立特里亚',
        'Spain':'西班牙',
        'Estonia':'爱沙尼亚',
        'Ethiopia':'埃塞俄比亚',
        'Finland':'芬兰',
        'Fiji':'斐济',
        'Falkland Islands':'福克兰群岛',
        'France':'法国',
        'Gabon':'加蓬',
        'United Kingdom':'英国',
        'Georgia':'格鲁吉亚',
        'Ghana':'加纳',
        'Guinea':'几内亚',
        'Gambia':'冈比亚',
        'Guinea Bissau':'几内亚比绍',
        'Eq. Guinea':'赤道几内亚',
        'Greece':'希腊',
        'Greenland':'格陵兰',
        'Guatemala':'危地马拉',
        'French Guiana':'法属圭亚那',
        'Guyana':'圭亚那',
        'Honduras':'洪都拉斯',
        'Croatia':'克罗地亚',
        'Haiti':'海地',
        'Hungary':'匈牙利',
        'Indonesia':'印度尼西亚',
        'India':'印度',
        'Ireland':'爱尔兰',
        'Iran':'伊朗',
        'Iraq':'伊拉克',
        'Iceland':'冰岛',
        'Israel':'以色列',
        'Italy':'意大利',
        'Jamaica':'牙买加',
        'Jordan':'约旦',
        'Japan':'日本',
        'Japan':'日本本土',
        'Kazakhstan':'哈萨克斯坦',
        'Kenya':'肯尼亚',
        'Kyrgyzstan':'吉尔吉斯斯坦',
        'Cambodia':'柬埔寨',
        'Korea':'韩国',
        'Kosovo':'科索沃',
        'Kuwait':'科威特',
        'Lao PDR':'老挝',
        'Lebanon':'黎巴嫩',
        'Liberia':'利比里亚',
        'Libya':'利比亚',
        'Sri Lanka':'斯里兰卡',
        'Lesotho':'莱索托',
        'Lithuania':'立陶宛',
        'Luxembourg':'卢森堡',
        'Latvia':'拉脱维亚',
        'Morocco':'摩洛哥',
        'Moldova':'摩尔多瓦',
        'Madagascar':'马达加斯加',
        'Mexico':'墨西哥',
        'Macedonia':'马其顿',
        'Mali':'马里',
        'Myanmar':'缅甸',
        'Montenegro':'黑山',
        'Mongolia':'蒙古',
        'Mozambique':'莫桑比克',
        'Mauritania':'毛里塔尼亚',
        'Malawi':'马拉维',
        'Malaysia':'马来西亚',
        'Namibia':'纳米比亚',
        'New Caledonia':'新喀里多尼亚',
        'Niger':'尼日尔',
        'Nigeria':'尼日利亚',
        'Nicaragua':'尼加拉瓜',
        'Netherlands':'荷兰',
        'Norway':'挪威',
        'Nepal':'尼泊尔',
        'New Zealand':'新西兰',
        'Oman':'阿曼',
        'Pakistan':'巴基斯坦',
        'Panama':'巴拿马',
        'Peru':'秘鲁',
        'Philippines':'菲律宾',
        'Papua New Guinea':'巴布亚新几内亚',
        'Poland':'波兰',
        'Puerto Rico':'波多黎各',
        'Dem. Rep. Korea':'朝鲜',
        'Portugal':'葡萄牙',
        'Paraguay':'巴拉圭',
        'Qatar':'卡塔尔',
        'Romania':'罗马尼亚',
        'Russia':'俄罗斯',
        'Rwanda':'卢旺达',
        'W. Sahara':'西撒哈拉',
        'Saudi Arabia':'沙特阿拉伯',
        'Sudan':'苏丹',
        'S. Sudan':'南苏丹',
        'Senegal':'塞内加尔',
        'Solomon Is.':'所罗门群岛',
        'Sierra Leone':'塞拉利昂',
        'El Salvador':'萨尔瓦多',
        'Somaliland':'索马里兰',
        'Somalia':'索马里',
        'Serbia':'塞尔维亚',
        'Suriname':'苏里南',
        'Slovakia':'斯洛伐克',
        'Slovenia':'斯洛文尼亚',
        'Sweden':'瑞典',
        'Swaziland':'斯威士兰',
        'Syria':'叙利亚',
        'Chad':'乍得',
        'Togo':'多哥',
        'Thailand':'泰国',
        'Tajikistan':'塔吉克斯坦',
        'Turkmenistan':'土库曼斯坦',
        'East Timor':'东帝汶',
        'Trinidad and Tobago':'特里尼达和多巴哥',
        'Tunisia':'突尼斯',
        'Turkey':'土耳其',
        'Tanzania':'坦桑尼亚',
        'Uganda':'乌干达',
        'Ukraine':'乌克兰',
        'Uruguay':'乌拉圭',
        'United States':'美国',
        'Uzbekistan':'乌兹别克斯坦',
        'Venezuela':'委内瑞拉',
        'Vietnam':'越南',
        'Vanuatu':'瓦努阿图',
        'West Bank':'西岸',
        'Yemen':'也门',
        'South Africa':'南非',
        'Zambia':'赞比亚',
        'Zimbabwe':'津巴布韦',
        'Maldives':'马尔代夫',
        'Mauritius':'毛里求斯',
        'Monaco':'摩纳哥',
        'Andorra':'安道尔',
        'Cape Verde':'佛得角',
        'San Marino':'圣马力诺',
        'Malta':'马耳他',
        'Comoros':'科摩罗',
        'Trinidad and Tobago':'特立尼达和多巴哥',
        'Barbados':'巴巴多斯',
        'Martinique':'马提尼克岛',
        'Saint Vincent and the Grenadines':'圣文森特和格林纳丁斯',
        'Antigua and Barbuda':'安提瓜和巴布达',
        'Grenada':'格林纳达',
        'Saint Lucia':'圣卢西亚',
        'Saint Kitts-Nevis':'圣基茨和尼维斯',
        'Seychelles':'塞舌尔',
        'Vatican City':'梵蒂冈'
    }

In [37]:
from pyecharts import options as opts
from pyecharts.charts import Map
import pandas as pd

def read_country_code():
    """
    获取国家中英文字典
    :return:
    """
    country_dict = {}
    for key, val in nameMap.items():  # 将 nameMap 列表里面键值互换
        country_dict[val] = key
    return country_dict

def read_csv():
    """
    读取数据，返回国家英文名称列表和累计确诊数列表
    :return:
    """
    country_dict = read_country_code()
    data = pd.read_csv("2019-nCoV.csv", index_col=False)

    countrys_names = list()
    confirmed_count = list()

    for x in range(len(data.index)):
        if data['name'].iloc[x] in country_dict.keys():
            countrys_names.append(country_dict[data['name'].iloc[x]])
            confirmed_count.append(int(data['confirm'].iloc[x]))
        else:
            print(data['name'].iloc[x])

    return countrys_names, confirmed_count

"""
绘制世界地图
"""
#获取国家数据和确诊数据
countrys_names, confirmed_count = read_csv()
def map_world_disease_dis() -> Map:
    c = (
        Map()
        .add(
            "确诊人数",
            [list(z) for z in zip(countrys_names, confirmed_count)],
            is_map_symbol_show=False,
            maptype="world",
            label_opts=opts.LabelOpts(is_show=True),
            itemstyle_opts=opts.ItemStyleOpts(color="rgb(49,60,72)")
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="全球 2019-nCoV 地图"),
            visualmap_opts=opts.VisualMapOpts(max_=1700000,is_show=True,),
        )
    )
    return c
map_world_disease_dis().render_notebook()

北马其顿
波黑
钻石号邮轮
列支敦士登公国
多米尼克


D:\soft\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
